In [31]:
from nba_api.live.nba.endpoints import scoreboard

# Today's Score Board
games = scoreboard.ScoreBoard()

# json
games.get_json()

# dictionary
games.get_dict()

{'meta': {'version': 1,
  'request': 'https://nba-prod-us-east-1-mediaops-stats.s3.amazonaws.com/NBA/liveData/scoreboard/todaysScoreboard_00.json',
  'time': '2023-11-11 08:40:57.4057',
  'code': 200},
 'scoreboard': {'gameDate': '2023-11-11',
  'leagueId': '00',
  'leagueName': 'National Basketball Association',
  'games': [{'gameId': '0022300175',
    'gameCode': '20231111/MIAATL',
    'gameStatus': 2,
    'gameStatusText': 'Half',
    'period': 2,
    'gameClock': 'PT00M00.00S',
    'gameTimeUTC': '2023-11-12T00:30:00Z',
    'gameEt': '2023-11-11T19:30:00-05:00',
    'regulationPeriods': 4,
    'ifNecessary': False,
    'seriesGameNumber': '',
    'seriesText': '',
    'seriesConference': '',
    'poRoundDesc': '',
    'gameSubtype': '',
    'homeTeam': {'teamId': 1610612737,
     'teamName': 'Hawks',
     'teamCity': 'Atlanta',
     'teamTricode': 'ATL',
     'wins': 5,
     'losses': 3,
     'score': 52,
     'seed': None,
     'inBonus': '0',
     'timeoutsRemaining': 3,
     'pe

In [32]:
#https://www.basketball-reference.com/about/ratings.html
def calculate_olivers_ortg(df):
    # For demonstration, I'm using only the needed columns and assuming Team level data
    df = df.copy()  # Create a copy to avoid modifying the original DataFrame
    
    # Adding additional columns for Oliver's calculations
    df['FG_Part'] = df['FGM'] * (1 - 0.5 * ((df['PTS'] - df['FTM']) / (2 * df['FGA'])))
    df['AST_Part'] = 0.5 * (((df['PTS'] - df['FTM']) / (2 * df['FGA'])) * df['AST'])
    df['FT_Part'] = (1 - (1 - (df['FTM'] / df['FTA']))**2) * 0.4 * df['FTA']
    df['Team_Scoring_Poss'] = df['FGM'] + (1 - (1 - (df['FTM'] / df['FTA']))**2) * df['FTA'] * 0.4
    df['Team_ORB_Weight'] = 1  # Assuming Team_ORB% = 1 for demonstration
    df['Team_Play%'] = df['Team_Scoring_Poss'] / (df['FGA'] + df['FTA'] * 0.4 + df['TOV'])
    
    # Calculating Scoring Possessions
    df['ScPoss'] = (df['FG_Part'] + df['AST_Part'] + df['FT_Part']) * (1 - (df['OREB'] / df['Team_Scoring_Poss']) * df['Team_ORB_Weight'] * df['Team_Play%'])
    
    # Calculating Missed FG and FT Possessions
    df['FGxPoss'] = (df['FGA'] - df['FGM']) * (1 - 1.07 * 1)  # Assuming Team_ORB% as 1 for simplification
    df['FTxPoss'] = (1 - (df['FTM'] / df['FTA'])**2) * 0.4 * df['FTA']
    
    # Calculating Total Possessions
    df['TotPoss'] = df['ScPoss'] + df['FGxPoss'] + df['FTxPoss'] + df['TOV']
    
    # For demonstration, assuming Individual Points Produced (PProd) is same as ScPoss
    df['PProd'] = df['ScPoss']
    
    # Calculating Oliver's Offensive Rating (ORtg)
    df['ORtg_Oliver'] = 100 * (df['PProd'] / df['TotPoss'])
    
    return df['ORtg_Oliver']  # Return the modified DataFrame including the ORtg_Oliver column

def calculate_olivers_drtg(df):
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Constants and Intermediate Variables for the Formula
    df['FMwt'] = (df['FG_PCT_OPP'] * (1 - df['REB_OPP'] / (df['REB_OPP'] + df['REB']))) / \
                 (df['FG_PCT_OPP'] * (1 - df['REB_OPP'] / (df['REB_OPP'] + df['REB'])) + \
                  (1 - df['FG_PCT_OPP']) * (df['REB_OPP'] / (df['REB_OPP'] + df['REB'])))
    
    # Calculation of Stops1
    df['Stops1'] = df['STL'] + df['BLK'] * df['FMwt'] * (1 - 1.07 * df['REB_OPP'] / (df['REB_OPP'] + df['REB'])) + \
                   df['DREB'] * (1 - df['FMwt'])
    
    # Calculation of Stops2
    df['Stops2'] = (((df['FGA_OPP'] - df['FGM_OPP'] - df['BLK']) / df['MIN']) * df['FMwt'] * \
                    (1 - 1.07 * df['REB_OPP'] / (df['REB_OPP'] + df['REB'])) + \
                    ((df['TOV_OPP'] - df['STL']) / df['MIN'])) * df['MIN'] + \
                   (df['PF'] / df['PF_OPP']) * 0.4 * df['FTA_OPP'] * \
                   (1 - (df['FTM_OPP'] / df['FTA_OPP']) ** 2)
    
    # Total Stops
    df['Stops'] = df['Stops1'] + df['Stops2']
    
    # Calculation of Defensive Possessions (already calculated in original function)
    # df['Defensive_Possessions'] = (df['FGA_OPP'] - df['OREB_OPP']) + df['TOV_OPP'] + (0.44 * df['FTA_OPP'])
    
    # Calculation of Stop%
    df['Stop%'] = (df['Stops'] * df['MIN_OPP']) / (df['Defensive_Possessions'] * df['MIN'])
    
    # Team's Defensive Rating
    df['Team_Defensive_Rating'] = 100 * (df['PTS_OPP'] / df['Defensive_Possessions'])
    
    # Individual Defensive Rating (DRtg)
    df['D_Pts_per_ScPoss'] = df['PTS_OPP'] / (df['FGM_OPP'] + (1 - (1 - (df['FTM_OPP'] / df['FTA_OPP'])) ** 2) * df['FTA_OPP'] * 0.4)
    df['DRtg_Oliver'] = df['Team_Defensive_Rating'] + 0.2 * (100 * df['D_Pts_per_ScPoss'] * (1 - df['Stop%']) - df['Team_Defensive_Rating'])
    
    return df[['DRtg_Oliver']]  # Return only the DRtg_Oliver column

def calculate_olivers_ortg_opp(df):
    # For demonstration, I'm using only the needed columns and assuming Team level data
    df = df.copy()  # Create a copy to avoid modifying the original DataFrame

    # Adding additional columns for Oliver's calculations
    # Swap the team stats with the opponent's stats
    df['FG_Part'] = df['FGM_opp'] * (1 - 0.5 * ((df['PTS_opp'] - df['FTM_opp']) / (2 * df['FGA_opp'])))
    df['AST_Part'] = 0.5 * (((df['PTS_opp'] - df['FTM_opp']) / (2 * df['FGA_opp'])) * df['AST_opp'])
    df['FT_Part'] = (1 - (1 - (df['FTM_opp'] / df['FTA_opp']))**2) * 0.4 * df['FTA_opp']
    df['Team_Scoring_Poss'] = df['FGM_opp'] + (1 - (1 - (df['FTM_opp'] / df['FTA_opp']))**2) * df['FTA_opp'] * 0.4

    df['Team_ORB_Weight'] = 1  # Assuming Team_ORB% = 1 for demonstration
    df['Team_Play%'] = df['Team_Scoring_Poss'] / (df['FGA_opp'] + df['FTA_opp'] * 0.4 + df['TOV_opp'])

    # Calculating Scoring Possessions
    df['ScPoss'] = (df['FG_Part'] + df['AST_Part'] + df['FT_Part']) * \
                    (1 - (df['OREB_opp'] / df['Team_Scoring_Poss']) * df['Team_ORB_Weight'] * df['Team_Play%'])

    # Calculating Missed FG and FT Possessions
    df['FGxPoss'] = (df['FGA_opp'] - df['FGM_opp']) * (1 - 1.07 * df['Team_ORB_Weight'])  # Assuming Team_ORB% as 1 for simplification
    df['FTxPoss'] = (1 - (df['FTM_opp'] / df['FTA_opp'])**2) * 0.4 * df['FTA_opp']

    # Calculating Total Possessions
    df['TotPoss'] = df['ScPoss'] + df['FGxPoss'] + df['FTxPoss'] + df['TOV_opp']

    # For demonstration, assuming Individual Points Produced (PProd) is same as ScPoss
    df['PProd'] = df['ScPoss']

    # Calculating Oliver's Offensive Rating (ORtg) from the opponent's perspective
    df['ORtg_Oliver'] = 100 * (df['PProd'] / df['TotPoss'])

    return df['ORtg_Oliver_OPP']  # Return the modified DataFrame including the ORtg_Oliver column


#Opposing Team Ratings for the future nba seasons summarized dataset
#https://www.basketball-reference.com/about/ratings.html
def calculate_olivers_drtg_opp(df):
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Constants and Intermediate Variables for the Formula
    # Swap the team stats with the opponent's stats for calculations
    df['FMwt'] = (df['FG_PCT'] * (1 - df['REB'] / (df['REB'] + df['REB_opp']))) / \
                 (df['FG_PCT'] * (1 - df['REB'] / (df['REB'] + df['REB_opp'])) + \
                  (1 - df['FG_PCT']) * (df['REB'] / (df['REB'] + df['REB_opp'])))
    
    # Calculation of Stops1
    df['Stops1'] = df['STL_opp'] + df['BLK_opp'] * df['FMwt'] * (1 - 1.07 * df['REB'] / (df['REB'] + df['REB_opp'])) + \
                   df['DREB_opp'] * (1 - df['FMwt'])
    
    # Calculation of Stops2
    df['Stops2'] = (((df['FGA'] - df['FGM'] - df['BLK_opp']) / df['MIN_opp']) * df['FMwt'] * \
                    (1 - 1.07 * df['REB'] / (df['REB'] + df['REB_opp'])) + \
                    ((df['TOV'] - df['STL_opp']) / df['MIN_opp'])) * df['MIN_opp'] + \
                   (df['PF_opp'] / df['PF']) * 0.4 * df['FTA'] * \
                   (1 - (df['FTM'] / df['FTA']) ** 2)
    
    # Total Stops
    df['Stops'] = df['Stops1'] + df['Stops2']
    
    # Calculation of Defensive Possessions using the team's offensive stats
    df['Defensive_Possessions'] = (df['FGA'] - df['OREB']) + df['TOV'] + (0.44 * df['FTA'])
    
    # Calculation of Stop%
    df['Stop%'] = (df['Stops'] * df['MIN']) / (df['Defensive_Possessions'] * df['MIN_opp'])
    
    # Team's Defensive Rating
    df['Team_Defensive_Rating'] = 100 * (df['PTS'] / df['Defensive_Possessions'])
    
    # Individual Defensive Rating (DRtg) from the opponent's perspective
    df['D_Pts_per_ScPoss'] = df['PTS'] / (df['FGM'] + (1 - (1 - (df['FTM'] / df['FTA'])) ** 2) * df['FTA'] * 0.4)
    df['DRtg_Oliver'] = df['Team_Defensive_Rating'] + 0.2 * (100 * df['D_Pts_per_ScPoss'] * (1 - df['Stop%']) - df['Team_Defensive_Rating'])
    
    return df[['DRtg_Oliver_OPP']]  # Return only the DRtg_Oliver column


In [33]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

def get_games_by_season(season_year_start):
    # Format the season_id to match the expected 'YYYY-ZZ' format
    season_id = f"{season_year_start}-{str(season_year_start+1)[-2:]}"
    try:
        gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable=season_id)
        games = gamefinder.get_data_frames()[0]
        return games
    except Exception as e:
        print(f"An error occurred for season {season_id}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on error

all_seasons_data = pd.DataFrame()

# Loop through the seasons (only data back to 1983-84 season)
for year in range(1986, 2024):  # Adjust the range as needed
    season_data = get_games_by_season(year)
    if not season_data.empty:
        all_seasons_data = pd.concat([all_seasons_data, season_data], ignore_index=True)
        #print(f"Data retrieved for {all_seasons_data['SEASON_ID'].nunique()} seasons. {all_seasons_data['SEASON_ID'].unique()} ")
    else:
        print(f"No data found for season starting in {year}")

# Check how many seasons were successfully retrieved
#print(f"Data retrieved for {all_seasons_data['SEASON_ID'].nunique()} seasons.")


#print(len(all_seasons_data))
#print(all_seasons_data.head()) 
#print list of unique team_name and team_id
print(all_seasons_data[['TEAM_NAME', 'TEAM_ID']].drop_duplicates().sort_values('TEAM_ID').reset_index(drop=True))
# Do something with all_seasons_data, like saving to a CSV file
# all_seasons_data.to_csv('all_seasons_data.csv', index=False)


                       TEAM_NAME     TEAM_ID
0    United States United States          40
1       Tel Aviv Maccabi Electra          41
2               China Team China          45
3        Puerto Rico Puerto Rico          54
4                    Team Mexico          65
..                           ...         ...
212              G League Ignite  1612709930
213        Mexico City Capitanes  1612709931
214   Ciudad de Mexico Capitanes  1612709931
215            Motor City Cruise  1612709932
216               Rip City Remix  1612709933

[217 rows x 2 columns]


In [34]:
current_teams = [1610612739, 1610612737, 1610612738, 1610612740, 1610612741, 1610612742, 1610612743, 1610612744, 1610612745, 1610612746, 1610612747, 1610612748, 1610612749, 1610612750, 1610612751, 1610612752, 1610612753, 1610612754, 1610612755, 1610612756, 1610612757, 1610612758, 1610612759, 1610612760, 1610612761, 1610612762, 1610612763, 1610612764, 1610612765, 1610612766]
#print(current_teams)

#serach for Teams to get their team id: in team_name column
#print(team_ids[team_ids['TEAM_NAME'] == 'Cleveland Cavaliers'])


# Filter the data to include only current NBA teams
all_seasons_data_filtered = all_seasons_data[all_seasons_data['TEAM_ID'].isin(current_teams)]

#sort data by game_date ascending
all_seasons_data_filtered = all_seasons_data_filtered.sort_values(by=["TEAM_ID", "GAME_DATE"], ascending=[True, False]).reset_index(drop=True)
# take only the first 
unique_teams = all_seasons_data_filtered.drop_duplicates(subset='TEAM_ID', keep='first')

#print(all_seasons_data_filtered[['TEAM_NAME', 'TEAM_ID']].drop_duplicates().sort_values('TEAM_ID').reset_index(drop=True))

# get unique list of team_id, team_name, and season_id as a copy so we can save it to data folder
team_ids = unique_teams[['TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION']].drop_duplicates().sort_values('TEAM_ID').reset_index(drop=True)
#print(team_ids)
unique_team_season_id = team_ids.to_csv(r'C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\ML\nba_w_l_prediction_models\nba_analysis\data\team_ids.csv', index=False)


# Function to determine home or away
def home_or_away(row):
    team = row['TEAM_ABBREVIATION']
    matchup = row['MATCHUP']
    
    if team + ' @ ' in matchup:
        return 'Away'
    elif team + ' vs. ' in matchup:
        return 'Home'
    else:
        return 'Unknown'
    
all_seasons_data_filtered['Home_Away'] = all_seasons_data_filtered.apply(home_or_away, axis=1)

#fix index
data = all_seasons_data_filtered.reset_index(drop=True)
print(len(data))
print(data.head())
#print(data.columns)

94934
  SEASON_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_NAME     GAME_ID  \
0     22023  1610612737               ATL  Atlanta Hawks  0022300172   
1     22023  1610612737               ATL  Atlanta Hawks  0022300155   
2     22023  1610612737               ATL  Atlanta Hawks  0022300135   
3     22023  1610612737               ATL  Atlanta Hawks  0022300117   
4     22023  1610612737               ATL  Atlanta Hawks  0022300104   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  OREB  DREB  REB  AST  STL  BLK  \
0  2023-11-09    ATL @ ORL  W  240  120  ...     8    28   36   22   13    7   
1  2023-11-06    ATL @ OKC  L  239  117  ...    25    34   59   27    8    3   
2  2023-11-04    ATL @ NOP  W  241  123  ...    19    33   52   28    4    6   
3  2023-11-01  ATL vs. WAS  W  241  130  ...    14    43   57   26    8    3   
4  2023-10-30  ATL vs. MIN  W  238  127  ...     4    32   36   28    6    7   

   TOV  PF  PLUS_MINUS  Home_Away  
0   17  25         1.0       Away  
1   15  

In [35]:
#MERGING GAME_ID's for _OPP data
# Assuming data is your original dataframe

# Splitting data into data_first and data_second
data_first = data#.iloc[::2].reset_index(drop=True)
data_second = data#.iloc[1::2].reset_index(drop=True)

# Drop unnecessary columns from data_second
columns_to_drop = ['GAME_DATE', 'MATCHUP', 'TEAM_ABBREVIATION']
data_second = data_second.drop(columns=columns_to_drop)

# Merge data_first and data_second
data = data_first.merge(data_second, on='GAME_ID', suffixes=('', '_OPP'))

# Ensure TEAM_NAME from data_first isn't the same as TEAM_NAME from data_second
data = data[data.TEAM_NAME != data.TEAM_NAME_OPP]

#***making into a 1 row per game format***
#group columns by group_id to get the first one of each group
#data = data.groupby('GAME_ID').first().reset_index()
#change WL column to show TEAM_ID for the winner and TEAM_ID_OPP for the loser
#data.loc[data['WL'] == 'W', 'WL'] = data['TEAM_ID']
#data.loc[data['WL'] == 'L', 'WL'] = data['TEAM_ID_OPP']


print(len(data))
print(data.head())


94806
  SEASON_ID     TEAM_ID TEAM_ABBREVIATION              TEAM_NAME     GAME_ID  \
1     22023  1610612737               ATL          Atlanta Hawks  0022300172   
2     22023  1610612753               ORL          Orlando Magic  0022300172   
5     22023  1610612737               ATL          Atlanta Hawks  0022300155   
6     22023  1610612760               OKC  Oklahoma City Thunder  0022300155   
9     22023  1610612737               ATL          Atlanta Hawks  0022300135   

    GAME_DATE      MATCHUP WL  MIN  PTS  ...  OREB_OPP  DREB_OPP  REB_OPP  \
1  2023-11-09    ATL @ ORL  W  240  120  ...        18        29       47   
2  2023-11-09  ORL vs. ATL  L  240  119  ...         8        28       36   
5  2023-11-06    ATL @ OKC  L  239  117  ...        11        38       49   
6  2023-11-06  OKC vs. ATL  W  240  126  ...        25        34       59   
9  2023-11-04    ATL @ NOP  W  241  123  ...         9        26       35   

   AST_OPP  STL_OPP  BLK_OPP  TOV_OPP  PF_OPP  PLU

In [36]:
# Create the unique matchup ID
def create_matchup_id(matchup):
    # Split the teams based on " vs. " or " @ "
    teams = matchup.split(' vs. ') if ' vs. ' in matchup else matchup.split(' @ ')
    # Sort the teams alphabetically and concatenate
    return ''.join(sorted(teams))

data['MATCHUP_ID'] = data['MATCHUP'].apply(create_matchup_id)
#This function ensures that no matter the order, the matchup will come out like this. Example below and can be used on the unseen data as well.
#data = data[data['MATCHUP_ID'] == 'HOUPOR']

#print(data.head())
print(len(data))


94806


In [37]:

#print('base_data = ',data.columns)
#print(data.head())
#stat addons list: PER, TS%, eFG%, 3PAr, FTr, ORB%, DRB%, TRB%, AST%, STL%, BLK%, TOV%, USG%, OWS, DWS, WS, WS/48, OBPM, DBPM, BPM, VORP
#adding TS%= PTS / (2 * (FGA + 0.44 * FTA))
data['TS%'] = data['PTS'] / (2 * (data['FGA'] + 0.44 * data['FTA']))



#**OFF Rating add
# Calculate offensive and defensive possessions
data['Offensive_Possessions'] = data['FGA'] + 0.44 * data['FTA'] + data['TOV'] - data['OREB']
#data['Defensive_Possessions'] = data['Opponent_FGA'] + 0.44 * data['Opponent_FTA'] + data['Opponent_TOV'] - data['Opponent_OREB']

# Calculate Offensive Rating and Defensive Rating
data['ORtg'] = (data['PTS'] / data['Offensive_Possessions']) * 100
#data['DRtg'] = (data['Points_Allowed'] / data['Defensive_Possessions']) * 100

#adding PER% = (PTS + REB + AST + STL + BLK - FGM - FTM - TOV) / MIN
data['PER'] = (data['PTS'] + data['REB'] + data['AST'] + data['STL'] + data['BLK'] -
             data['FGM'] - data['FTM'] - data['TOV'])/ data['MIN']
#standardized to 48 minutes
#data['PER%'] = (data['PER'] / data['MIN']) * 48

#adding PER% = (PTS + REB + AST + STL + BLK - FGM - FTM - TOV) / OP
data['PER%'] = ((data['PTS'] + data['REB'] + data['AST'] + data['STL'] + data['BLK'] -
             data['FGM'] - data['FTM'] - data['TOV']) / data['Offensive_Possessions']) * 100

# add per min function
def per_min(col):
    # Create a new column name based on the input column's name
    column_pm = col.name[0] + 'PM'
    
    # Set the new column's values to the per-minute values of the input column
    data[column_pm] = col / data['MIN']

#add points per minute  
data['PTS_PER_MIN'] = data['PTS'] / data['MIN']

#adding rebounds per minute
#data['RPM'] = data['REB'] / data['MIN']
per_min(data['REB'])
per_min(data['OREB'])
per_min(data['DREB'])

#adding assists per minute
#data['APM'] = data['AST'] / data['MIN']
per_min(data['AST'])

#adding steals per minute
#data['SPM'] = data['STL'] / data['MIN']
per_min(data['STL'])

#adding blocks per minute
#data['BPM'] = data['BLK'] / data['MIN']
per_min(data['BLK'])

#adding turnovers per minute
#data['TPM'] = data['TOV'] / data['MIN']
per_min(data['TOV'])

#adding fouls per minute
#data['FPM'] = data['PF'] / data['MIN']
per_min(data['PF'])

#adding eFG%
data['eFG%'] = (data['FGM'] + 0.5 * data['FG3M']) / data['FGA']

#adding AST%
data['AST%'] = data['AST'] / data['FGM']

#adding USG%
data['USG%'] = (data['FGA'] + 0.44 * data['FTA'] + data['TOV']) / data['MIN']
#add all the percentages to the list of columns

#_OPP stats:
data['PTS_PER_MIN_OPP'] = data['PTS_OPP'] / data['MIN_OPP']
# Calculate FG, FG3, and FT percentages
data['FG_PCT_OPP'] = data['FGM_OPP'] / data['FGA_OPP']
data['FG3_PCT_OPP'] = data['FG3M_OPP'] / data['FG3A_OPP']
data['FT_PCT_OPP'] = data['FTM_OPP'] / data['FTA_OPP']

# Calculate additional stats
data['TS%_OPP'] = data['PTS_OPP'] / (2 * (data['FGA_OPP'] + 0.44 * data['FTA_OPP']))
data['eFG%_OPP'] = (data['FGM_OPP'] + 0.5 * data['FG3M_OPP']) / data['FGA_OPP']
data['AST%_OPP'] = (data['AST_OPP'] / data['FGA_OPP']) * 100

# Calculate Defensive possessions _OPP
data['Defensive_Possessions'] = (data['FGA_OPP'] - data['OREB_OPP']) + data['TOV_OPP'] + (0.44 * data['FTA_OPP'])

# Calculate Defensive Rating (DRtg)
data['DRtg'] = (data['PTS_OPP'] / data['Defensive_Possessions']) * 100

data['DPER%'] = ((data['PTS_OPP'] + data['REB_OPP'] + data['AST_OPP'] + data['STL_OPP'] + data['BLK_OPP'] -
                        data['FGM_OPP'] - data['FTM_OPP'] - data['TOV_OPP']) / data['Defensive_Possessions']) * 100

#difference between team and _opp stats
data['FG_PCT_DIFF'] = data['FG_PCT_OPP'] - data['FG_PCT']
data['FG3_PCT_DIFF'] = data['FG3_PCT_OPP'] - data['FG3_PCT']
data['FT_PCT_DIFF'] = data['FT_PCT_OPP'] - data['FT_PCT']
data['TS%_DIFF'] = data['TS%_OPP'] - data['TS%']
data['eFG%_DIFF'] = data['eFG%_OPP'] - data['eFG%']
data['AST%_DIFF'] = data['AST%_OPP'] - data['AST%']
data['ORtg_DIFF'] = data['DRtg'] - data['ORtg']
data['PER%_DIFF'] = data['DPER%'] - data['PER%']
data['PTS_DIFF'] = data['PTS'] - data['PTS_OPP']
data['PTS_PER_MIN_DIFF'] = data['PTS_PER_MIN'] - data['PTS_PER_MIN_OPP']

#drop all _opp stats
#data = data.drop(columns=['FG_PCT_OPP', 'FG3_PCT_OPP', 'FT_PCT_OPP', 'TS%_OPP', 'eFG%_OPP', 'AST%_OPP', 'Defensive_Possessions',
#                            'DRtg', 'DPER%', 'PLUS_MINUS_OPP'])

#***PRE PROCESSING***
#*Date transformations
# Convert the date column to datetime format
data['GAME_DATE'] = pd.to_datetime(data['GAME_DATE'])

# Extract numerical features from the date column
data['YEAR'] = data['GAME_DATE'].dt.year
data['MONTH'] = data['GAME_DATE'].dt.month
data['DAY'] = data['GAME_DATE'].dt.day

season_data = data
print(season_data.head())

  SEASON_ID     TEAM_ID TEAM_ABBREVIATION              TEAM_NAME     GAME_ID  \
1     22023  1610612737               ATL          Atlanta Hawks  0022300172   
2     22023  1610612753               ORL          Orlando Magic  0022300172   
5     22023  1610612737               ATL          Atlanta Hawks  0022300155   
6     22023  1610612760               OKC  Oklahoma City Thunder  0022300155   
9     22023  1610612737               ATL          Atlanta Hawks  0022300135   

   GAME_DATE      MATCHUP WL  MIN  PTS  ...  TS%_DIFF  eFG%_DIFF  AST%_DIFF  \
1 2023-11-09    ATL @ ORL  W  240  120  ... -0.046482  -0.048366  20.574526   
2 2023-11-09  ORL vs. ATL  L  240  119  ...  0.046482   0.048366  25.429972   
5 2023-11-06    ATL @ OKC  L  239  117  ...  0.101108   0.112015  28.012878   
6 2023-11-06  OKC vs. ATL  W  240  126  ... -0.101108  -0.112015  25.870588   
9 2023-11-04    ATL @ NOP  W  241  123  ... -0.030796  -0.035330  27.949206   

   ORtg_DIFF  PER%_DIFF  PTS_DIFF  PTS_PER_M

In [38]:
#change this to earlier in the code and pull in the preprocessing for each of the models

# add in offensive and defensive ratings with correlation matrix to wins and losses
# Calculate Oliver's Defensive Rating (ORtg), it adds on 'DRtg_Oliver', 'Stops', 'Stop%', 'Team_Defensive_Rating', 'D_Pts_per_ScPoss', 'DRtg_Oliver'
season_data['DRtg_Oliver'] = calculate_olivers_drtg(season_data)
#print(season_data.head())

# Calculate Oliver's Offensive Rating (ORtg)
season_data['ORtg_Oliver'] = calculate_olivers_ortg(season_data)

#print(ortg_season_data)
#print ortg_season_data columns
#print(ortg_season_data.columns)
#season_data['ORtg_Oliver'] = ortg_season_data['ORtg_Oliver']
#print(season_data.head())
print(len(season_data))
#print(season_data.head())
#MERGING GAME_ID's for _OPP data
# Assuming data is your original dataframe

# Splitting data into data_first and data_second
data_first = season_data#.iloc[::2].reset_index(drop=True)
data_second = season_data#.iloc[1::2].reset_index(drop=True)

#only include necessary columns in data_second being GAME_ID, TEAM_ID, TEAM_NAME, WL_encoded, WL, ORtg_Oliver, DRtg_Oliver
data_second = data_second[['GAME_ID', 'TEAM_NAME', 'ORtg_Oliver', 'DRtg_Oliver']]
print(data_second.columns)

# Merge data_first and data_second
data = data_first.merge(data_second, on='GAME_ID', suffixes=('', '_OPP2'))

# Take down the difference between the two teams' ORtg and DRtg
data['ORtg_Oliver_DIFF'] = data['ORtg_Oliver'] - data['ORtg_Oliver_OPP2']
data['DRtg_Oliver_DIFF'] = data['DRtg_Oliver'] - data['DRtg_Oliver_OPP2']
# Ensure TEAM_NAME from data_first isn't the same as TEAM_NAME from data_second
data = data[data.TEAM_NAME != data.TEAM_NAME_OPP2]
data = data[data.TEAM_NAME_OPP == data.TEAM_NAME_OPP2]

#drop team_name_opp2
data = data.drop(columns=['TEAM_NAME_OPP2'])

#rename _opp2 columns to _opp
data = data.rename(columns={'ORtg_Oliver_OPP2': 'ORtg_Oliver_OPP', 'DRtg_Oliver_OPP2': 'DRtg_Oliver_OPP'})

#WL_encoded 0 = "W" and 1 = "L"
data['WL_encoded'] = data['WL'].replace('W', 0)
data['WL_encoded'] = data['WL_encoded'].replace('L', 1)

data['WL_encoded_OPP'] = data['WL_OPP'].replace('W', 0)
data['WL_encoded_OPP'] = data['WL_encoded_OPP'].replace('L', 1)

#drop plus_minus and plus_minus_opp
data = data.drop(columns=['PLUS_MINUS', 'PLUS_MINUS_OPP'])

print(len(data))
print(data.columns)


94806
Index(['GAME_ID', 'TEAM_NAME', 'ORtg_Oliver', 'DRtg_Oliver'], dtype='object')
94832
Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'Home_Away', 'SEASON_ID_OPP',
       'TEAM_ID_OPP', 'TEAM_NAME_OPP', 'WL_OPP', 'MIN_OPP', 'PTS_OPP',
       'FGM_OPP', 'FGA_OPP', 'FG_PCT_OPP', 'FG3M_OPP', 'FG3A_OPP',
       'FG3_PCT_OPP', 'FTM_OPP', 'FTA_OPP', 'FT_PCT_OPP', 'OREB_OPP',
       'DREB_OPP', 'REB_OPP', 'AST_OPP', 'STL_OPP', 'BLK_OPP', 'TOV_OPP',
       'PF_OPP', 'Home_Away_OPP', 'MATCHUP_ID', 'TS%', 'Offensive_Possessions',
       'ORtg', 'PER', 'PER%', 'PTS_PER_MIN', 'RPM', 'OPM', 'DPM', 'APM', 'SPM',
       'BPM', 'TPM', 'PPM', 'eFG%', 'AST%', 'USG%', 'PTS_PER_MIN_OPP',
       'TS%_OPP', 'eFG%_OPP', 'AST%_OPP', 'Defensive_Possessions', 'DRtg',
       'DPER%', 'FG

In [39]:
import numpy as np

# Find out which columns have NaN or infinite values and the number of such values in each column
nan_or_inf_values = data.replace([np.inf, -np.inf], np.nan).isnull().sum()
nan_or_inf_columns_before = nan_or_inf_values[nan_or_inf_values > 0]

print("Columns with NaN or infinite values and their counts before dropping:")
print(nan_or_inf_columns_before)

# Replace infinite values with NaNs and then drop rows with NaN values
data = data.replace([np.inf, -np.inf], np.nan).dropna()

# Check NaN or infinite columns again after dropping
nan_or_inf_values_after = data.isnull().sum()
nan_or_inf_columns_after = nan_or_inf_values_after[nan_or_inf_values_after > 0]

print("\nColumns with NaN or infinite values and their counts after dropping:")
print(nan_or_inf_columns_after)

# Print the earliest game_date after dropping NaN and infinite values
min_game_date = data['GAME_DATE'].min()
print("\nEarliest GAME_DATE after dropping NaNs and infinites:", min_game_date)


Columns with NaN or infinite values and their counts before dropping:
WL                   18
FG_PCT               10
FG3_PCT             307
FT_PCT                8
WL_OPP               18
FG_PCT_OPP           10
FG3_PCT_OPP         307
FT_PCT_OPP            8
TS%                   8
ORtg                  6
PER                   4
PER%                  6
PTS_PER_MIN           4
RPM                   4
OPM                   4
DPM                   4
APM                   4
SPM                   4
BPM                   4
TPM                   4
PPM                   4
eFG%                 10
AST%                 12
USG%                  4
PTS_PER_MIN_OPP       4
TS%_OPP               8
eFG%_OPP             10
AST%_OPP             10
DRtg                  6
DPER%                 6
FG_PCT_DIFF          20
FG3_PCT_DIFF        604
FT_PCT_DIFF          16
TS%_DIFF             16
eFG%_DIFF            20
AST%_DIFF            22
ORtg_DIFF            12
PER%_DIFF            12
PTS_PER_MIN_DIFF  

In [40]:
#sort season_data by team_id and game_date
data = data.sort_values(by=["TEAM_ID", "GAME_DATE"], ascending=[True, False]).reset_index(drop=True)
print(data.head())

#filter data for just the start_date = pd.to_datetime('1979-10-01') end_date = pd.to_datetime('2024-06-30')
#last_to_curr_season_data = season_data[season_data.GAME_DATE <= '2023-06-30']
#last_to_curr_season_data = season_data[season_data.GAME_DATE >= '2022-10-01']

# Save the data to a CSV file
#season_data.to_csv(r'C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\ML\nba_w_l_prediction_models\nba_analysis\data\nba_prepreprocess_data.csv', index=False)

#filter data for just the start_date = pd.to_datetime('1979-10-01') end_date = pd.to_datetime('2024-06-30')
#three_pt_era_data = season_data[season_data.GAME_DATE <= '2023-06-30']
#three_pt_era_data = season_data[season_data.GAME_DATE >= '1979-10-01']

# Save the data to a CSV file
data.to_csv(r'C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\data\nba\model_preprocessing_store\nba_threeptera_prepreprocess_data.csv', index=False)



  SEASON_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_NAME     GAME_ID  \
0     22023  1610612737               ATL  Atlanta Hawks  0022300172   
1     22023  1610612737               ATL  Atlanta Hawks  0022300155   
2     22023  1610612737               ATL  Atlanta Hawks  0022300135   
3     22023  1610612737               ATL  Atlanta Hawks  0022300117   
4     22023  1610612737               ATL  Atlanta Hawks  0022300104   

   GAME_DATE      MATCHUP WL  MIN  PTS  ...  MONTH  DAY  DRtg_Oliver  \
0 2023-11-09    ATL @ ORL  W  240  120  ...     11    9   112.268180   
1 2023-11-06    ATL @ OKC  L  239  117  ...     11    6   122.591646   
2 2023-11-04    ATL @ NOP  W  241  123  ...     11    4   114.854321   
3 2023-11-01  ATL vs. WAS  W  241  130  ...     11    1   111.242429   
4 2023-10-30  ATL vs. MIN  W  238  127  ...     10   30   120.735965   

   ORtg_Oliver  ORtg_Oliver_OPP  DRtg_Oliver_OPP  ORtg_Oliver_DIFF  \
0    69.813182        64.957567       114.968081          4.8556

In [41]:

#data adds: playoff indicator, championship indicator, conference indicator, division indicator 


import pandas as pd

def prepare_data(df):
    # Sum the statistics by team and season_id
    sum_data = df.groupby(['TEAM_ID', 'Home_Away', 'MATCHUP_ID']).agg({
        'PTS': 'mean',
        'FGM': 'mean',
        'FGA': 'mean',
        'FG3M': 'mean',
        'FG3A': 'mean',
        'FTM': 'mean',
        'FTA': 'mean',
        'AST': 'mean',
        'OREB': 'mean',
        'TOV': 'mean',
        'STL': 'mean',
        'BLK': 'mean',
        'REB': 'mean',
        'MIN': 'mean',
        'DREB': 'mean',
        #'PLUS_MINUS': 'mean',  # Mean of the PLUS_MINUS remains as is
        'ORtg_Oliver': 'mean',
        'DRtg_Oliver': 'mean'

        , 'PTS_OPP': 'mean'
        , 'FGM_OPP': 'mean'
        , 'FGA_OPP': 'mean'
        , 'FG3M_OPP': 'mean'
        , 'FG3A_OPP': 'mean'
        , 'FTM_OPP': 'mean'
        , 'FTA_OPP': 'mean'
        , 'AST_OPP': 'mean'
        , 'OREB_OPP': 'mean'
        , 'DREB_OPP': 'mean'
        , 'TOV_OPP': 'mean'
        , 'STL_OPP': 'mean'
        , 'BLK_OPP': 'mean'
        , 'REB_OPP': 'mean'
        , 'MIN_OPP': 'mean'
        #, 'PLUS_MINUS_OPP': 'mean'  # Mean of the PLUS_MINUS remains as is
        , 'ORtg_Oliver_OPP': 'mean'
        , 'DRtg_Oliver_OPP': 'mean'
    }).reset_index()
    
    #points per minute
    sum_data['PTS_PER_MIN'] = sum_data['PTS'] / sum_data['MIN']

    # Calculate FG, FG3, and FT percentages
    sum_data['FG_PCT'] = sum_data['FGM'] / sum_data['FGA']
    sum_data['FG3_PCT'] = sum_data['FG3M'] / sum_data['FG3A']
    sum_data['FT_PCT'] = sum_data['FTM'] / sum_data['FTA']
    
    # Calculate additional stats
    sum_data['TS%'] = sum_data['PTS'] / (2 * (sum_data['FGA'] + 0.44 * sum_data['FTA']))
    sum_data['eFG%'] = (sum_data['FGM'] + 0.5 * sum_data['FG3M']) / sum_data['FGA']
    sum_data['AST%'] = (sum_data['AST'] / sum_data['FGA']) * 100
    
    # Calculate offensive possessions: https://kenpom.com/blog/the-possession/
    sum_data['Offensive_Possessions'] = (sum_data['FGA'] - sum_data['OREB']) + sum_data['TOV'] + (0.44 * sum_data['FTA'])

    # Calculate Offensive Rating (ORtg) and Player Efficiency Rating (PER)
    sum_data['ORtg'] = (sum_data['PTS'] / sum_data['Offensive_Possessions']) * 100
    # Offensive Team Rating (https://www.basketball-reference.com/about/ratings.html) = (Players Points*Total FG%) + Opponents Differential= 1/5 of possessions - Times Fouled+ FTM* FT% * OAPOW (Official Adjusted Players Offensive Withstand)

    sum_data['PER%'] = ((sum_data['PTS'] + sum_data['REB'] + sum_data['AST'] + sum_data['STL'] + sum_data['BLK'] -
                         sum_data['FGM'] - sum_data['FTM'] - sum_data['TOV']) / sum_data['Offensive_Possessions']) * 100
    
    #offensirve efficiency
    sum_data['OFF_EFF'] = (sum_data['PTS'] + sum_data['AST'] + sum_data['OREB']) / sum_data['MIN']


    #_OPP stats:
    #points per minute
    sum_data['PTS_PER_MIN_OPP'] = sum_data['PTS_OPP'] / sum_data['MIN_OPP']
    # Calculate FG, FG3, and FT percentages
    sum_data['FG_PCT_OPP'] = sum_data['FGM_OPP'] / sum_data['FGA_OPP']
    sum_data['FG3_PCT_OPP'] = sum_data['FG3M_OPP'] / sum_data['FG3A_OPP']
    sum_data['FT_PCT_OPP'] = sum_data['FTM_OPP'] / sum_data['FTA_OPP']
    
    # Calculate additional stats
    sum_data['TS%_OPP'] = sum_data['PTS_OPP'] / (2 * (sum_data['FGA_OPP'] + 0.44 * sum_data['FTA_OPP']))
    sum_data['eFG%_OPP'] = (sum_data['FGM_OPP'] + 0.5 * sum_data['FG3M_OPP']) / sum_data['FGA_OPP']
    sum_data['AST%_OPP'] = (sum_data['AST_OPP'] / sum_data['FGA_OPP']) * 100

    # Calculate Defensive possessions _OPP
    sum_data['Defensive_Possessions'] = (sum_data['FGA_OPP'] - sum_data['OREB_OPP']) + sum_data['TOV_OPP'] + (0.44 * sum_data['FTA_OPP'])

    # Calculate Defensive Rating (DRtg)
    sum_data['DRtg'] = (sum_data['PTS_OPP'] / sum_data['Defensive_Possessions']) * 100

    sum_data['DPER%'] = ((sum_data['PTS_OPP'] + sum_data['REB_OPP'] + sum_data['AST_OPP'] + sum_data['STL_OPP'] + sum_data['BLK_OPP'] -
                         sum_data['FGM_OPP'] - sum_data['FTM_OPP'] - sum_data['TOV_OPP']) / sum_data['Defensive_Possessions']) * 100
    

    #difference between team and _opp stats
    sum_data['PTS_DIFF'] = sum_data['PTS'] - sum_data['PTS_OPP']
    sum_data['PTS_PER_MIN_DIFF'] = sum_data['PTS_PER_MIN'] - sum_data['PTS_PER_MIN_OPP']
    sum_data['FG_PCT_DIFF'] = sum_data['FG_PCT_OPP'] - sum_data['FG_PCT']
    sum_data['FG3_PCT_DIFF'] = sum_data['FG3_PCT_OPP'] - sum_data['FG3_PCT']
    sum_data['FT_PCT_DIFF'] = sum_data['FT_PCT_OPP'] - sum_data['FT_PCT']
    sum_data['TS%_DIFF'] = sum_data['TS%_OPP'] - sum_data['TS%']
    sum_data['eFG%_DIFF'] = sum_data['eFG%_OPP'] - sum_data['eFG%']
    sum_data['AST%_DIFF'] = sum_data['AST%_OPP'] - sum_data['AST%']
    sum_data['ORtg_DIFF'] = sum_data['DRtg'] - sum_data['ORtg']
    sum_data['PER%_DIFF'] = sum_data['DPER%'] - sum_data['PER%']
    sum_data['ORtg_Oliver_DIFF'] = sum_data['ORtg_Oliver'] - sum_data['ORtg_Oliver_OPP']
    sum_data['DRtg_Oliver_DIFF'] = sum_data['DRtg_Oliver'] - sum_data['DRtg_Oliver_OPP']

    #drop all _opp stats
    #sum_data = sum_data.drop(columns=['FG_PCT_OPP', 'FG3_PCT_OPP', 'FT_PCT_OPP', 'TS%_OPP', 'eFG%_OPP', 'AST%_OPP', 'Defensive_Possessions',
                               #'DRtg', 'DPER%', 'DRtg_Oliver_OPP', 'ORtg_Oliver_OPP']) #, 'PLUS_MINUS_OPP'

    


    return sum_data

# filter the data for the 2022-2023 NBA season to now
#could also use season_id = 42022
start_date = pd.to_datetime('2022-10-24')
end_date = pd.to_datetime('2024-06-30')
future_predictor_data = data[(data['GAME_DATE'] >= start_date) & (data['GAME_DATE'] <= end_date)].copy()
print(future_predictor_data.head())

prepared_data = prepare_data(future_predictor_data)
#print(prepared_data.head())


data_drop_columns = ['PTS', 'FGM', 'FG3M', 'FTM', 'FTA', 'PTS', 'REB', 'OREB', 'FGA', 'FG3A', 'OREB', 'ORtg_Oliver', 'DRtg_Oliver',
                                    'AST', 'TOV', 'STL', 'BLK',  'Offensive_Possessions', 'SEASON_ID'
                                    'PTS_OPP', 'FGM_OPP', 'FG3M_OPP', 'FTM_OPP', 'FTA_OPP', 'PTS_OPP', 'REB_OPP', 'OREB_OPP', 'FGA_OPP', 'FG3A_OPP', 'OREB_OPP', 
                                    'AST_OPP', 'TOV_OPP', 'STL_OPP', 'BLK_OPP',  'Defensive_Possessions', 'SEASON_ID_OPP', 'Home_Away_OPP', 'PTS_PER_MIN_OPP'] #, 'DREB'

#export the data dataset without the dropped columns
future_season_data_stats = prepared_data#[[col for col in prepared_data.columns if col not in data_drop_columns]]

# export X to csv to apply averages onto future nba season data on a long short-term basis
future_season_data_stats.to_csv(r'C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\ML\nba_w_l_prediction_models\nba_analysis\data\future_season_data_stats.csv', index=False)
print(future_season_data_stats.columns)
print(future_season_data_stats.head())
print(len(future_season_data_stats))

  SEASON_ID     TEAM_ID TEAM_ABBREVIATION      TEAM_NAME     GAME_ID  \
0     22023  1610612737               ATL  Atlanta Hawks  0022300172   
1     22023  1610612737               ATL  Atlanta Hawks  0022300155   
2     22023  1610612737               ATL  Atlanta Hawks  0022300135   
3     22023  1610612737               ATL  Atlanta Hawks  0022300117   
4     22023  1610612737               ATL  Atlanta Hawks  0022300104   

   GAME_DATE      MATCHUP WL  MIN  PTS  ...  MONTH  DAY  DRtg_Oliver  \
0 2023-11-09    ATL @ ORL  W  240  120  ...     11    9   112.268180   
1 2023-11-06    ATL @ OKC  L  239  117  ...     11    6   122.591646   
2 2023-11-04    ATL @ NOP  W  241  123  ...     11    4   114.854321   
3 2023-11-01  ATL vs. WAS  W  241  130  ...     11    1   111.242429   
4 2023-10-30  ATL vs. MIN  W  238  127  ...     10   30   120.735965   

   ORtg_Oliver  ORtg_Oliver_OPP  DRtg_Oliver_OPP  ORtg_Oliver_DIFF  \
0    69.813182        64.957567       114.968081          4.8556

In [42]:
# Filter data since the beginning of the season
current_season_prediction_tracker = data[data['GAME_DATE'] >= '2023-10-24']

current_season_prediction_tracker.to_csv(r'C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\ML\nba_w_l_prediction_models\nba_analysis\data\23_24_current_season_prediction_tracker.csv', index=False)
#print(current_season_prediction_tracker.columns)
current_season_prediction_tracker = current_season_prediction_tracker[current_season_prediction_tracker['TEAM_NAME'] == 'Los Angeles Lakers']
print(current_season_prediction_tracker.head())
print(len(current_season_prediction_tracker))
#get max date from the GAME_DATE column
print(current_season_prediction_tracker['GAME_DATE'].max())

      SEASON_ID     TEAM_ID TEAM_ABBREVIATION           TEAM_NAME     GAME_ID  \
31434     22023  1610612747               LAL  Los Angeles Lakers  0022300015   
31435     22023  1610612747               LAL  Los Angeles Lakers  0022300163   
31436     22023  1610612747               LAL  Los Angeles Lakers  0022300150   
31437     22023  1610612747               LAL  Los Angeles Lakers  0022300134   
31438     22023  1610612747               LAL  Los Angeles Lakers  0022300127   

       GAME_DATE      MATCHUP WL  MIN  PTS  ...  MONTH  DAY  DRtg_Oliver  \
31434 2023-11-10    LAL @ PHX  W  240  122  ...     11   10   118.306022   
31435 2023-11-08    LAL @ HOU  L  241   94  ...     11    8   131.892275   
31436 2023-11-06    LAL @ MIA  L  239  107  ...     11    6   110.277081   
31437 2023-11-04    LAL @ ORL  L  239  101  ...     11    4   117.028422   
31438 2023-11-01  LAL vs. LAC  W  266  130  ...     11    1   113.201410   

       ORtg_Oliver  ORtg_Oliver_OPP  DRtg_Oliver_OPP  OR

In [43]:
# Filter data since the beginning of the season
current_season_prediction_tracker = data[data['GAME_DATE'] >= '2022-10-24']

current_season_prediction_tracker.to_csv(r'C:\Users\ghadf\OneDrive\Desktop\Data Analytics\Python\data\nba\model_preprocessing_store\22_23_team_non_tree_data.csv', index=False)
print(current_season_prediction_tracker.columns)

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'Home_Away', 'SEASON_ID_OPP',
       'TEAM_ID_OPP', 'TEAM_NAME_OPP', 'WL_OPP', 'MIN_OPP', 'PTS_OPP',
       'FGM_OPP', 'FGA_OPP', 'FG_PCT_OPP', 'FG3M_OPP', 'FG3A_OPP',
       'FG3_PCT_OPP', 'FTM_OPP', 'FTA_OPP', 'FT_PCT_OPP', 'OREB_OPP',
       'DREB_OPP', 'REB_OPP', 'AST_OPP', 'STL_OPP', 'BLK_OPP', 'TOV_OPP',
       'PF_OPP', 'Home_Away_OPP', 'MATCHUP_ID', 'TS%', 'Offensive_Possessions',
       'ORtg', 'PER', 'PER%', 'PTS_PER_MIN', 'RPM', 'OPM', 'DPM', 'APM', 'SPM',
       'BPM', 'TPM', 'PPM', 'eFG%', 'AST%', 'USG%', 'PTS_PER_MIN_OPP',
       'TS%_OPP', 'eFG%_OPP', 'AST%_OPP', 'Defensive_Possessions', 'DRtg',
       'DPER%', 'FG_PCT_DIFF', 'FG3_PCT_DIFF', 'FT_PCT_DIFF', 'TS%_DIFF',
       'eFG%_DIFF', 'AST%_DIFF', 'O